# ImageNet 1K Data Quality


In [1]:
import lance
import duckdb
import torchvision
import torch
import pandas as pd

In [2]:
%load_ext sql
%sql duckdb:///:memory:

{}


In [3]:
uri = "imagenet.lance"

ds = lance.dataset(uri)

In [4]:
ds.schema

id: int32
image: extension<image[binary]<ImageBinaryType>>
label: int16
name: dictionary<values=string, indices=int16, ordered=0>
split: dictionary<values=string, indices=int8, ordered=0>

In [5]:
ds

In [6]:
# %%sql --lance

# SELECT image, label FROM ds LIMIT 5

In [7]:
%%sql

SELECT split, count(split) FROM ds GROUP BY split

Took 0.006806373596191406


,split,count(split)
0,train,10000
1,test,10000
2,validation,10000


In [8]:
from torchvision.models import resnet50, vit_b_16

resnet = resnet50(weights="DEFAULT").cuda()
vit = vit_b_16(weights="DEFAULT").cuda()

In [9]:
# TODO: make easy conversion between lance.Dataset and pytorch.Dataset

from lance.pytorch import Dataset
dataset = Dataset(uri, columns=["id", "image"], mode="batch")
print(dataset.schema)

id: int32
image: extension<image[binary]<ImageBinaryType>>
label: int16
name: dictionary<values=string, indices=int16, ordered=0>
split: dictionary<values=string, indices=int8, ordered=0>


In [10]:
transform = torchvision.models.ResNet50_Weights.DEFAULT.transforms()
dataset = Dataset(
    uri, 
    columns=["id", "image"],
    mode="batch",
    batch_size=128
)
results = []
with torch.no_grad():
    resnet.eval()
    for batch in dataset:
        imgs = [transform(img).cuda() for img in batch[1]]
        prediction = resnet(torch.stack(imgs)).squeeze(0).softmax(0)
        topk = torch.topk(prediction, 2)
        for pk, scores, indices in zip(
            batch[0], topk.values.tolist(), topk.indices.tolist()
        ):
            results.append({
                "id": pk.item(),
                "resnet": {
                    "label": indices[0], 
                    "score": scores[0], 
                    "second_label": indices[1],  # Secondary guess
                    "second_score": scores[1],  # Confidence of the secondary guess.
                }
            })
            
df = pd.DataFrame(data=results)
df.astype({"id": "int32"})
df

,id,resnet
0,1,"{'label': 726, 'score': 0.694125771522522, 'se..."
1,2,"{'label': 917, 'score': 0.8421049118041992, 's..."
2,3,"{'label': 13, 'score': 0.8555712103843689, 'se..."
3,4,"{'label': 939, 'score': 0.9331550002098083, 's..."
4,5,"{'label': 6, 'score': 0.9336700439453125, 'sec..."
...,...,...
29995,29996,"{'label': 658, 'score': 0.9771027565002441, 's..."
29996,29997,"{'label': 433, 'score': 0.9925998449325562, 's..."
29997,29998,"{'label': 47, 'score': 0.9799998998641968, 'se..."
29998,29999,"{'label': 801, 'score': 0.9724040627479553, 's..."


In [11]:
import pyarrow as pa
df = df.astype({"id": "int32"})
tab = pa.Table.from_pandas(df)


In [12]:
ds = ds.merge(tab, left_on="id", right_on="id")

In [13]:
ds.schema

id: int32
image: extension<image[binary]<ImageBinaryType>>
label: int16
name: dictionary<values=string, indices=int16, ordered=0>
split: dictionary<values=string, indices=int8, ordered=0>
resnet: struct<label: int64, score: double, second_label: int64, second_score: double>
  child 0, label: int64
  child 1, score: double
  child 2, second_label: int64
  child 3, second_score: double

In [14]:
ds = lance.dataset(uri)

In [15]:
ds.schema

id: int32
image: extension<image[binary]<ImageBinaryType>>
label: int16
name: dictionary<values=string, indices=int16, ordered=0>
split: dictionary<values=string, indices=int8, ordered=0>
resnet: struct<label: int64, score: double, second_label: int64, second_score: double>
  child 0, label: int64
  child 1, score: double
  child 2, second_label: int64
  child 3, second_score: double

In [18]:
%%sql

SELECT * EXCLUDE(image) FROM ds WHERE label != resnet.label AND split != 'test' LIMIT 10


Took 0.010018348693847656


,id,label,name,split,resnet
0,13,575,"golfcart, golf cart",train,"{'label': 785, 'score': 0.3907656967639923, 's..."
1,16,219,"cocker spaniel, English cocker spaniel, cocker",train,"{'label': 213, 'score': 0.8196388483047485, 's..."
2,45,968,cup,train,"{'label': 899, 'score': 0.4809149503707886, 's..."
3,52,304,"leaf beetle, chrysomelid",train,"{'label': 305, 'score': 0.8165855407714844, 's..."
4,54,940,spaghetti squash,train,"{'label': 936, 'score': 0.70237797498703, 'sec..."
5,60,67,"diamondback, diamondback rattlesnake, Crotalus...",train,"{'label': 68, 'score': 0.8087248802185059, 'se..."
6,71,459,"brassiere, bra, bandeau",train,"{'label': 824, 'score': 0.8276260495185852, 's..."
7,79,782,"screen, CRT screen",train,"{'label': 664, 'score': 0.8757317066192627, 's..."
8,86,256,"Newfoundland, Newfoundland dog",train,"{'label': 205, 'score': 0.6166163086891174, 's..."
9,105,371,"patas, hussar monkey, Erythrocebus patas",train,"{'label': 370, 'score': 0.8230134844779968, 's..."


In [17]:
duckdb.query("SELECT id, label from ds LIMIT 5").arrow()

pyarrow.Table
id: int32
label: int16
----
id: [[1,2,3,4,5]]
label: [[726,917,13,939,6]]